# RoBERTa Finetuning for Arousal Detection

This Jupyter Notebook is dedicated to fine-tuning a model for detecting arousal.

In [1]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


### Imports

In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from transformers import TrainingArguments
from sklearn.model_selection import KFold
import numpy as np

In [4]:
df = pd.read_csv('..Dataset/Training_Emotions.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('fake', axis=1)
df.head()

,id,tweet,valence,arousal
0,126727,"Andrew Brigden MP is clearly a virologist, sta...",negative,medium
1,98481,@RepMattGaetz I do not understand How trump lo...,negative,medium
2,45066,Only four nations allow elective abortions aft...,negative,low
3,62614,@brhodes Agreed. It's time to stop demonizing ...,negative,medium
4,15038,@shortiemagee New York State passed a law mand...,positive,low


In [5]:
len(df)

49986

### Prepare Dataset

In [6]:
def arousal_vectorized(arousal_series):
    arousal_map = {'high': 0, 'medium': 1, 'low': 2}
    arousal_values = arousal_series.map(arousal_map)
    if arousal_values.isnull().any() :
        raise ValueError("Invalid arousal category found")
    return arousal_values

In [7]:
def decode_vectorized(series):
    arousal_map = {0: 'High', 1: 'Medium', 2: 'Low'}
    arousal = series.map(arousal_map)
    return pd.DataFrame({'arousal': arousal})

In [8]:
df = df[df['arousal'] != 'neutral']
df['label'] = arousal_vectorized(df['arousal'])
df = df.rename(columns={'tweet': 'text'})
df['label'] = df['label'].astype(int)

<ipython-input-8-d9bdff6bed18>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = arousal_vectorized(df['arousal'])


In [9]:
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

### Define Model

In [10]:
ROBERTA = "FacebookAI/xlm-roberta-base"
def tokenizer(checkpoint):
    return AutoTokenizer.from_pretrained(checkpoint)

tokenizer_RoBERTa_uncased = tokenizer(ROBERTA)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def tokenize_function(batch, tokenizer):
    return tokenizer(batch["text"].tolist(), truncation=True)

In [12]:
data_collator_RoBERTa_uncased = DataCollatorWithPadding(tokenizer=tokenizer_RoBERTa_uncased, return_tensors="pt")

In [13]:
model_RoBERTa_uncased = AutoModelForSequenceClassification.from_pretrained(ROBERTA, num_labels=3, ignore_mismatched_sizes=True)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")

    return {"precision": precision, "recall": recall, "acc": acc, "f1": f1}

In [14]:
fold_results = []
model_name = 'Arousal_Model'
batch_size = 8
num_labels = 3
for fold, (train_index, val_index) in enumerate(kf.split(train_val_df), 1):
    print(f"Training Fold {fold}")

    # Train Val
    train_df = train_val_df.iloc[train_index]
    val_df = train_val_df.iloc[val_index]
    train_df = train_df.drop(columns=['id'])
    val_df = val_df.drop(columns=['id'])

    # Tokenize
    tokenized_datasets_RoBERTa_uncased = {
        "train": Dataset.from_dict(tokenize_function(train_df, tokenizer_RoBERTa_uncased)),
        "val": Dataset.from_dict(tokenize_function(val_df, tokenizer_RoBERTa_uncased))
    }

    # Add labels
    tokenized_datasets_RoBERTa_uncased["train"] = tokenized_datasets_RoBERTa_uncased["train"].add_column("labels", train_df["label"].tolist())
    tokenized_datasets_RoBERTa_uncased["val"] = tokenized_datasets_RoBERTa_uncased["val"].add_column("labels", val_df["label"].tolist())


    # Init Folds
    model_RoBERTa_uncased = AutoModelForSequenceClassification.from_pretrained(ROBERTA, num_labels=num_labels)


    logging_steps = len(tokenized_datasets_RoBERTa_uncased["train"]) // batch_size


    training_args = TrainingArguments(
        output_dir=f"{model_name}_fold_{fold}",
        num_train_epochs=3,
        save_strategy="epoch",
        save_total_limit=1,
        learning_rate=1e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        disable_tqdm=False,
        logging_steps=logging_steps,
        log_level="info",
        label_smoothing_factor=0.1,
        weight_decay=0.005,
        lr_scheduler_type='cosine',
        warmup_steps=200
    )

    # Trainers
    trainer = Trainer(
        model=model_RoBERTa_uncased,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=tokenized_datasets_RoBERTa_uncased["train"],
        eval_dataset=tokenized_datasets_RoBERTa_uncased["val"],
        data_collator=data_collator_RoBERTa_uncased,
        tokenizer=tokenizer_RoBERTa_uncased,
    )

    trainer.train()

    eval_results = trainer.evaluate()
    fold_results.append(eval_results)

    print(f"Fold {fold} Results:", eval_results)
    print("\n")

avg_results = {key: np.mean([res[key] for res in fold_results]) for key in fold_results[0].keys()}
print("Average Results across all Folds:", avg_results)

Training Fold 1


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
***** Running training *****
  Num examples = 35,919
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13,470
  Number of trainable parameters = 278,045,955


Epoch,Training Loss,Validation Loss,Precision,Recall,Acc,F1
1,0.857700,0.777817,0.679358,0.678168,0.694098,0.678681
2,0.726500,0.770544,0.699651,0.690655,0.706459,0.694398
3,0.644900,0.821051,0.691835,0.693519,0.704454,0.692619



***** Running Evaluation *****
  Num examples = 8980
  Batch size = 8
Saving model checkpoint to Arousal_Model_fold_1/checkpoint-4490
Configuration saved in Arousal_Model_fold_1/checkpoint-4490/config.json
Model weights saved in Arousal_Model_fold_1/checkpoint-4490/model.safetensors
tokenizer config file saved in Arousal_Model_fold_1/checkpoint-4490/tokenizer_config.json
Special tokens file saved in Arousal_Model_fold_1/checkpoint-4490/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 8980
  Batch size = 8
Saving model checkpoint to Arousal_Model_fold_1/checkpoint-8980
Configuration saved in Arousal_Model_fold_1/checkpoint-8980/config.json
Model weights saved in Arousal_Model_fold_1/checkpoint-8980/model.safetensors
tokenizer config file saved in Arousal_Model_fold_1/checkpoint-8980/tokenizer_config.json
Special tokens file saved in Arousal_Model_fold_1/checkpoint-8980/special_tokens_map.json
Deleting older checkpoint [Arousal_Model_fold_1/checkpoint-4490] due t

Fold 1 Results: {'eval_loss': 0.8210505843162537, 'eval_precision': 0.6918345634162694, 'eval_recall': 0.6935194215283736, 'eval_acc': 0.7044543429844098, 'eval_f1': 0.6926186547782058, 'eval_runtime': 17.0013, 'eval_samples_per_second': 528.194, 'eval_steps_per_second': 66.054, 'epoch': 3.0}


Training Fold 2


...

Fold 4 Results: {'eval_loss': 0.8212875127792358, 'eval_precision': 0.6954665499520113, 'eval_recall': 0.6956888986793052, 'eval_acc': 0.7076837416481069, 'eval_f1': 0.6952942854417078, 'eval_runtime': 16.9356, 'eval_samples_per_second': 530.245, 'eval_steps_per_second': 66.31, 'epoch': 3.0}


Training Fold 5


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--xlm-roberta-base/snapshots/e73636d4f797dec63c3081bb6ed5c7b0bb3f2089/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_

Epoch,Training Loss,Validation Loss,Precision,Recall,Acc,F1
1,0.859600,0.854815,0.648787,0.668693,0.666555,0.642533
2,0.723900,0.780231,0.699497,0.700427,0.708654,0.699939
3,0.639400,0.815782,0.694815,0.693960,0.705424,0.694191



***** Running Evaluation *****
  Num examples = 8979
  Batch size = 8
Saving model checkpoint to Arousal_Model_fold_5/checkpoint-4490
Configuration saved in Arousal_Model_fold_5/checkpoint-4490/config.json
Model weights saved in Arousal_Model_fold_5/checkpoint-4490/model.safetensors
tokenizer config file saved in Arousal_Model_fold_5/checkpoint-4490/tokenizer_config.json
Special tokens file saved in Arousal_Model_fold_5/checkpoint-4490/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 8979
  Batch size = 8
Saving model checkpoint to Arousal_Model_fold_5/checkpoint-8980
Configuration saved in Arousal_Model_fold_5/checkpoint-8980/config.json
Model weights saved in Arousal_Model_fold_5/checkpoint-8980/model.safetensors
tokenizer config file saved in Arousal_Model_fold_5/checkpoint-8980/tokenizer_config.json
Special tokens file saved in Arousal_Model_fold_5/checkpoint-8980/special_tokens_map.json
Deleting older checkpoint [Arousal_Model_fold_5/checkpoint-4490] due t

Fold 5 Results: {'eval_loss': 0.8157823085784912, 'eval_precision': 0.6948147857727024, 'eval_recall': 0.6939601804337058, 'eval_acc': 0.7054237665664328, 'eval_f1': 0.694190559373259, 'eval_runtime': 16.7761, 'eval_samples_per_second': 535.225, 'eval_steps_per_second': 66.94, 'epoch': 3.0}


Average Results across all Folds: {'eval_loss': 0.8188462615013122, 'eval_precision': 0.6940424319502734, 'eval_recall': 0.695253139370721, 'eval_acc': 0.706296334605046, 'eval_f1': 0.6945097928600482, 'eval_runtime': 16.834200000000003, 'eval_samples_per_second': 533.4498, 'eval_steps_per_second': 66.7122, 'epoch': 3.0}


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
model_name = 'Arousal_Model'
batch_size = 8
num_labels = 3


print("Training Final Model on Entire Dataset")

full_train_df = train_val_df.drop(columns=['id'])

train_data, val_data = train_test_split(full_train_df, test_size=0.1, random_state=42)


tokenized_dataset_full = Dataset.from_dict(tokenize_function(train_data, tokenizer_RoBERTa_uncased))
tokenized_dataset_full = tokenized_dataset_full.add_column("labels", train_data["label"].tolist())

tokenized_dataset_eval = Dataset.from_dict(tokenize_function(val_data, tokenizer_RoBERTa_uncased))
tokenized_dataset_eval = tokenized_dataset_eval.add_column("labels", val_data["label"].tolist())

logging_steps = len(tokenized_dataset_full) // batch_size


final_model = AutoModelForSequenceClassification.from_pretrained(ROBERTA, num_labels=num_labels)

final_training_args = TrainingArguments(
    output_dir=f"{model_name}_final",
        num_train_epochs=3,
        learning_rate=1e-5,
        save_strategy="epoch",
        save_total_limit=1,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        disable_tqdm=False,
        logging_steps=logging_steps,
        log_level="info",
        label_smoothing_factor=0.1,
        weight_decay=0.005,
        lr_scheduler_type='cosine',
        warmup_steps=200
)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    compute_metrics = compute_metrics,
    train_dataset=tokenized_dataset_full,
    eval_dataset=tokenized_dataset_eval,  # Add this line
    tokenizer=tokenizer_RoBERTa_uncased,
)

final_trainer.train()

final_trainer.save_model(f"{model_name}_final")

print("Final Model Training Completed")

print("Evaluating Final Model on Test Set")

test_df = test_df.drop(columns=['id'])  
tokenized_test_dataset = Dataset.from_dict(tokenize_function(test_df, tokenizer_RoBERTa_uncased))
tokenized_test_dataset = tokenized_test_dataset.add_column("labels", test_df["label"].tolist())

test_results = final_trainer.evaluate(eval_dataset=tokenized_test_dataset)

print("Test Set Evaluation Results:")
for key, value in test_results.items():
    print(f"{key}: {value}")

predictions = final_trainer.predict(test_dataset=tokenized_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

Training Final Model on Entire Dataset


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
***** Running training *****
  Num examples = 40,409
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15,156
  Number of trainable parameters = 278,045,955


Epoch,Training Loss,Validation Loss,Precision,Recall,Acc,F1
1,0.851400,0.773476,0.687412,0.668025,0.691314,0.675129
2,0.724700,0.787285,0.698178,0.703322,0.707127,0.699606
3,0.642700,0.813517,0.701059,0.698579,0.712027,0.699745



***** Running Evaluation *****
  Num examples = 4490
  Batch size = 8
Saving model checkpoint to Arousal_Model_final/checkpoint-5052
Configuration saved in Arousal_Model_final/checkpoint-5052/config.json
Model weights saved in Arousal_Model_final/checkpoint-5052/model.safetensors
tokenizer config file saved in Arousal_Model_final/checkpoint-5052/tokenizer_config.json
Special tokens file saved in Arousal_Model_final/checkpoint-5052/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 4490
  Batch size = 8
Saving model checkpoint to Arousal_Model_final/checkpoint-10104
Configuration saved in Arousal_Model_final/checkpoint-10104/config.json
Model weights saved in Arousal_Model_final/checkpoint-10104/model.safetensors
tokenizer config file saved in Arousal_Model_final/checkpoint-10104/tokenizer_config.json
Special tokens file saved in Arousal_Model_final/checkpoint-10104/special_tokens_map.json
Deleting older checkpoint [Arousal_Model_final/checkpoint-5052] due to args

Final Model Training Completed
Evaluating Final Model on Test Set



***** Running Evaluation *****
  Num examples = 4989
  Batch size = 8



***** Running Prediction *****
  Num examples = 4989
  Batch size = 8


Test Set Evaluation Results:
eval_loss: 0.8146065473556519
eval_precision: 0.6980631147344236
eval_recall: 0.6958022464074146
eval_acc: 0.70695530166366
eval_f1: 0.6968023368824791
eval_runtime: 9.5776
eval_samples_per_second: 520.902
eval_steps_per_second: 65.152
epoch: 3.0
